In [1]:
import pandas as pd
from sqlalchemy import create_engine
import requests
import time
import os
from pandas.io.json import json_normalize
#importing modules 

In [2]:
#START CODE HERE#
#EXTRACT#
x = ['capsules',
     'cores',
     'dragons',
     'history',
     'info',
     'landpads',
     'launches',
     'launchpads',
     'missions',
     'payloads',
     'rockets',
     'roadster',
     'ships']
#url links to extract all data from SpaceX API website

collection_df = pd.DataFrame(x,columns=['collection'])

#creating a dataframe to loop through

In [3]:
#creating an empty list to loop through and append each API call.
spacex_all = []
for x in collection_df['collection']:
    url = 'https://api.spacexdata.com/v3/'+x
    response = requests.get(url).json()
    spacex_all.append(response)

In [4]:
collection_names = [x for x in collection_df['collection'] + '_df']
    
capsules_df = pd.DataFrame(spacex_all[0])
cores_df=pd.DataFrame(spacex_all[1])
dragons_df=pd.DataFrame(spacex_all[2])
history_df=pd.DataFrame(spacex_all[3])
Info_df=pd.DataFrame(spacex_all[4])
landpads_df=pd.DataFrame(spacex_all[5])
launches_df=pd.DataFrame(spacex_all[6])
launchpads_df=pd.DataFrame(spacex_all[7])
missions_df=pd.DataFrame(spacex_all[8])
payloads_df=pd.DataFrame(spacex_all[9])
rockets_df=pd.DataFrame(spacex_all[10])s
roadster_df=pd.DataFrame(spacex_all[11])
ships_df=pd.DataFrame(spacex_all[12])

#assigning each API call to their own DataFrame for Transformation

In [5]:
#TRANSFORM
#definding some methods to trasnform columns with different date format.

def convertdate(x):
    x = pd.to_datetime(x,format='%Y-%m-%d')
    return x

def convertunix(x):
    x = pd.to_datetime(x,unit='s')
    return x

In [6]:
#Capsules Data Transformation

capsules_df = capsules_df[['capsule_id','capsule_serial','details','original_launch','landings','missions','type','reuse_count','status']].copy()
capsules_df['original_launch'] = capsules_df['original_launch'].apply(convertdate)
capsules_df['mission_name'] = capsules_df['missions'][0:14].apply(lambda x: x[0]['name'])
capsules_df['mission_flight'] = capsules_df['missions'][0:14].apply(lambda x: x[0]['flight'])
capsules_df = capsules_df.drop(columns='missions')
#capsules df ready for import

In [7]:
#Cores Data Transformation

cores_df=cores_df[['core_serial','details','missions','asds_attempts','asds_landings',
                    'block','original_launch','reuse_count','rtls_attempts','rtls_landings','water_landing',
                    'status']].copy()
cores_df['mission_name'] = cores_df['missions'][0:62].apply(lambda x: x[0]['name'])
cores_df['mission_flight'] = cores_df['missions'][0:62].apply(lambda x: x[0]['flight'])
cores_df['block'] = cores_df['block'].fillna(value='0')
cores_df['original_launch'] = cores_df['original_launch'].apply(convertdate)
cores_df=cores_df[['core_serial','details','mission_name','mission_flight','asds_attempts','asds_landings',
                    'block','original_launch','reuse_count','rtls_attempts','rtls_landings','water_landing',
                    'status']].copy()
#cores_df is ready for export

In [8]:
#Launches Data Transformation

launches_df = launches_df[['flight_number','mission_name','mission_id','upcoming','launch_year','launch_date_unix',\
                          'is_tentative','tentative_max_precision','rocket','ships','launch_site','launch_success',\
                          'launch_failure_details','links','details','static_fire_date_utc']].copy()
launches_df['launch_date_unix'] = launches_df['launch_date_unix'].apply(convertdate)
launches_df['static_fire_date_utc'] = launches_df['static_fire_date_utc'].apply(convertdate)
launches_df = launches_df.drop(columns=['rocket','launch_site','launch_failure_details','links'])
launch_df = launches_df.rename(columns={'launch_date_unix':'launch_date'})
launch_df = launches_df.rename(columns={'static_fire_date_utc':'static_fire_date'})

In [9]:
#Missions Data Transformation

missions_df=missions_df[['mission_id','mission_name','description','manufacturers','payload_ids','twitter','website','wikipedia']].copy()

In [15]:
#Payloads Data Transformation

payloads_df=payloads_df[['payload_id','payload_type','cap_serial','cargo_manifest','customers',
                         'flight_time_sec','manufacturer','mass_returned_kg','nationality',
                         'norad_id','orbit','payload_mass_kg','reused']].copy()


In [11]:
#Create Connection String to SQL Server (pgAdmin)

connection_string = 'postgres:root123@localhost:5432/mission_spacex'
engine = create_engine(f'postgresql://{connection_string}')

In [12]:
#load the finalized transformation data into SQL Server

missions_df.to_sql(name='missions',con=engine,if_exists='replace',index=True)

In [17]:
payloads_df.to_sql(name='payloads',con=engine,if_exists='replace',index=True)

In [18]:
capsules_df.to_sql(name='capsules', con=engine,if_exists='replace', index=False)

In [19]:
launches_df.to_sql(name='launches',con=engine,if_exists='replace',index=False)

In [21]:
cores_df.to_sql(name='cores',con=engine,if_exists='replace',index=False)